In [1]:
import pandas as pd
import pickle
import hashlib
import os
os.chdir('Resources/')

In [2]:
def power(a, b, c):
    x = 1
    y = a
    while b > 0:
        if b % 2 != 0:
            x = (x * y) % c
        y = (y * y) % c
        b = int(b / 2)
    return x % c

In [3]:
def decrypt(en_msg, p, key, q):
    dr_msg = []
    h = power(p, key, q)
    for i in range(0, len(en_msg)):
        dr_msg.append(int(en_msg[i] / h))  # Decrypt each element
    return dr_msg

In [4]:
with open('3_Private_Key.pkl', 'rb') as f_private:
    private_key_data = pickle.load(f_private)

q = private_key_data['q']
private_keys = private_key_data['private_keys']

In [5]:
with open('4_Encrypted_Values_and_P.pkl', 'rb') as f_encrypted:
    encrypted_data = pickle.load(f_encrypted)

encrypted_values_list = encrypted_data['encrypted_values_list']
p_list = encrypted_data['p_list']

In [6]:
df = pd.read_csv('1_Structured_Data.csv')

In [7]:
for i in range(11):
    decrypted_values = []

    for encrypted_val, p_value in zip(encrypted_values_list[i], p_list[i]):
        if encrypted_val is not None and p_value is not None:
            decrypted_value = decrypt([encrypted_val], p_value, private_keys[i], q)
            decrypted_values.append(decrypted_value[0])  # Decrypt returns a list, so take the first element
        else:
            decrypted_values.append(None)

    attribute_values = df.iloc[:, i].tolist()

    _, labels = pd.factorize(attribute_values)

    decoded_values = []
    for decrypted_value in decrypted_values:
        if decrypted_value is not None:
            decoded_values.append(labels[decrypted_value])  
        else:
            decoded_values.append(None)  

    print(f"Attribute {i+1} - Private Key (key): {private_keys[i]}")
    
    mismatch_count = 0
    
    for row_num, (decrypted_value, original_value) in enumerate(zip(decoded_values, attribute_values), start=1):
        if decrypted_value is None and original_value is None:
            continue
        if (decrypted_value is None and pd.isna(original_value)) or (pd.isna(decrypted_value) and original_value is None):
            continue
        
        if decrypted_value != original_value:
            print(f"Attribute {i+1} - Mismatch at Row {row_num}: Decrypted Value = {decrypted_value}, Original Value = {original_value}")
            mismatch_count += 1
       
    print(f"Attribute {i+1} - Total Mismatches: {mismatch_count}")
    print("\n" + "-"*50 + "\n")

C:\Users\USER\AppData\Local\Temp\ipykernel_13436\563586268.py:13: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  _, labels = pd.factorize(attribute_values)


Attribute 1 - Private Key (key): 6012834814088712862906120093983930950844580058895
Attribute 1 - Total Mismatches: 0

--------------------------------------------------

Attribute 2 - Private Key (key): 14017283848122657760267277998402180888335564595295
Attribute 2 - Total Mismatches: 0

--------------------------------------------------

Attribute 3 - Private Key (key): 8257378963198520837442387125559233697514460487698
Attribute 3 - Total Mismatches: 0

--------------------------------------------------

Attribute 4 - Private Key (key): 16037647293930807176387058042006233682891549347698
Attribute 4 - Total Mismatches: 0

--------------------------------------------------

Attribute 5 - Private Key (key): 11094550728020179905585589860523236879470944710850
Attribute 5 - Total Mismatches: 0

--------------------------------------------------

Attribute 6 - Private Key (key): 86597721845167548703828182318833449338186891534
Attribute 6 - Total Mismatches: 0

-------------------------------